## 시가총액 / Tobin's Q

In [1]:
import pandas as pd
import requests
import json
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
import time
time.sleep(0.3)
from pykrx import stock
pd.options.display.float_format = '{:.6f}'.format

In [2]:
cdf = pd.read_csv('data/company_name_and_code.csv', index_col = 0)
cdf

,corp_code,corp_name,stock_code
0,365387,AJ네트웍스,95570
1,125080,AK홀딩스,6840
2,219097,BGF,27410
3,1263022,BGF리테일,282330
4,858364,BNK금융지주,138930
...,...,...,...
726,111421,휴니드테크놀러지스,5870
727,362238,휴비스,79980
728,156488,휴스틸,5010
729,103176,흥국화재,540


In [3]:
df_2019 = stock.get_market_cap("20191230")
df_2019 = df_2019.reset_index()
df_2019 = df_2019.drop(['종가', '거래량', '거래대금', '상장주식수'], axis = 1)
df_2019.columns = ['stock_code', 'market_cap']
df_2019

,stock_code,market_cap
0,005930,333113866290000
1,000660,68505022546500
2,005935,37359056180000
3,035420,30737698167500
4,207940,28649445000000
...,...,...
2470,245450,2701731600
2471,276240,2587200000
2472,001529,2552590900
2473,238500,2224028850


In [4]:
df_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2475 entries, 0 to 2474
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   stock_code  2475 non-null   object
 1   market_cap  2475 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 38.8+ KB


In [5]:
cdf.loc[:, 'stock_code'] = cdf['stock_code'].astype(str).str.strip().str.zfill(6)
cdf['stock_code']

C:\Users\USER\AppData\Local\Temp\ipykernel_14532\3437129154.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['095570' '006840' '027410' '282330' '138930' '001460' '001040' '079160'
 '000120' '011150' '097950' '000590' '012030' '005830' '016610' '000990'
 '015590' '000210' '007340' '004840' '155660' '069730' '092780' '017940'
 '007700' '114090' '078930' '006360' '001250' '007070' '012630' '039570'
 '089470' '294870' '009540' '267250' '267270' '071970' '010620' '042670'
 '267260' '097230' '014790' '003580' '204320' '060980' '011200' '035000'
 '002460' '298050' '015360' '175330' '234080' '001060' '096760' '008970'
 '105560' '009440' '119650' '092220' '016380' '001390' '033180' '001940'
 '025000' '092230' '000040' '044450' '058850' '058860' '093050' '003550'
 '034220' '051900' '032640' '011070' '066570' '037560' '051910' '079550'
 '006260' '000680' '229640' '108320' '001120' '108670' '023150' '035420'
 '181710' '00

0      095570
1      006840
2      027410
3      282330
4      138930
        ...  
726    005870
727    079980
728    005010
729    000540
730    003280
Name: stock_code, Length: 731, dtype: object

In [6]:
mdf = cdf[cdf['stock_code'].astype(str).str.strip().isin(df_2019['stock_code'].astype(str).str.strip())]
mdf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 730 entries, 0 to 730
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   corp_code   730 non-null    int64 
 1   corp_name   730 non-null    object
 2   stock_code  730 non-null    object
dtypes: int64(1), object(2)
memory usage: 22.8+ KB


In [7]:
mdf = mdf.merge(
    right = df_2019,
    on = 'stock_code',
    how = 'left')
mdf

,corp_code,corp_name,stock_code,market_cap
0,365387,AJ네트웍스,095570,232706806150
1,125080,AK홀딩스,006840,458365610600
2,219097,BGF,027410,536014029600
3,1263022,BGF리테일,282330,2929622067000
4,858364,BNK금융지주,138930,2496663984360
...,...,...,...,...
725,111421,휴니드테크놀러지스,005870,102623429050
726,362238,휴비스,079980,210105000000
727,156488,휴스틸,005010,76064246640
728,103176,흥국화재,000540,196903706925


In [8]:
def market_cap(cdf, start_year = 2019, end_year = 2024):
    result_lst = []

    for year in range(start_year, end_year +1):
        if year == 2023:
            df = stock.get_market_cap(f"{year}1228")
            
        elif year == 2022:
            df = stock.get_market_cap(f"{year}1229")
            
        else:
            df = stock.get_market_cap(f"{year}1230")
        
        df = df.reset_index()
        df = df.drop(['종가', '거래량', '거래대금', '상장주식수'], axis = 1)
        df.columns = ['stock_code', 'market_cap']
        
        df = cdf.merge(right = df, on = 'stock_code', how = 'left').fillna(0)
        
        result_lst.append(df)
        
    return result_lst

In [9]:
rdf_2019, rdf_2020, rdf_2021, rdf_2022, rdf_2023, rdf_2024 = market_cap(cdf, 2019, 2024)

In [10]:
rdf_2019

,corp_code,corp_name,stock_code,market_cap
0,365387,AJ네트웍스,095570,232706806150.000000
1,125080,AK홀딩스,006840,458365610600.000000
2,219097,BGF,027410,536014029600.000000
3,1263022,BGF리테일,282330,2929622067000.000000
4,858364,BNK금융지주,138930,2496663984360.000000
...,...,...,...,...
726,111421,휴니드테크놀러지스,005870,102623429050.000000
727,362238,휴비스,079980,210105000000.000000
728,156488,휴스틸,005010,76064246640.000000
729,103176,흥국화재,000540,196903706925.000000


In [11]:
rdf_lst = [rdf_2019, rdf_2020, rdf_2021, rdf_2022, rdf_2023, rdf_2024]

for year, df in zip(range(2019, 2025), rdf_lst):
    df.to_csv(f'data/market_cap/market_cap_{year}.csv')

In [12]:
temp_lst = []
for year in range(2019, 2025):
    temp_lst.append(pd.read_csv(f'data/Major_Shareholder/Major_Shareholder_correction/MSE_C_{year}.csv', index_col = 0))

cadf_2019, cadf_2020, cadf_2021, cadf_2022, cadf_2023, cadf_2024 = temp_lst

In [13]:
cadf_2019

,corp_code,corp_name,year,total_asset,asset_size,total_debt,debt_ratio
0,365387,AJ네트웍스,2019,1803255225986.000000,28.220615,1455914070366.000000,0.807381
1,125080,AK홀딩스,2019,4327964555085.000000,29.096118,2898025666994.000000,0.669605
2,219097,BGF,2019,1657381460248.000000,28.136260,129126735585.000000,0.077910
3,1263022,BGF리테일,2019,2172410899706.000000,28.406859,1549815137631.000000,0.713408
4,858364,BNK금융지주,2019,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...
726,111421,휴니드테크놀러지스,2019,260965121649.000000,26.287653,145116828409.000000,0.556077
727,362238,휴비스,2019,877170812678.000000,27.499968,481045043591.000000,0.548405
728,156488,휴스틸,2019,579096387394.000000,27.084735,182538348103.000000,0.315212
729,103176,흥국화재,2019,0.000000,0.000000,0.000000,0.000000


In [14]:
tobin_2019 = cadf_2019.merge(rdf_2019, how = 'inner', on = ['corp_code', 'corp_name']).drop('stock_code', axis = 1)
tobin_2019

,corp_code,corp_name,year,total_asset,asset_size,total_debt,debt_ratio,market_cap
0,365387,AJ네트웍스,2019,1803255225986.000000,28.220615,1455914070366.000000,0.807381,232706806150.000000
1,125080,AK홀딩스,2019,4327964555085.000000,29.096118,2898025666994.000000,0.669605,458365610600.000000
2,219097,BGF,2019,1657381460248.000000,28.136260,129126735585.000000,0.077910,536014029600.000000
3,1263022,BGF리테일,2019,2172410899706.000000,28.406859,1549815137631.000000,0.713408,2929622067000.000000
4,858364,BNK금융지주,2019,0.000000,0.000000,0.000000,0.000000,2496663984360.000000
...,...,...,...,...,...,...,...,...
726,111421,휴니드테크놀러지스,2019,260965121649.000000,26.287653,145116828409.000000,0.556077,102623429050.000000
727,362238,휴비스,2019,877170812678.000000,27.499968,481045043591.000000,0.548405,210105000000.000000
728,156488,휴스틸,2019,579096387394.000000,27.084735,182538348103.000000,0.315212,76064246640.000000
729,103176,흥국화재,2019,0.000000,0.000000,0.000000,0.000000,196903706925.000000


In [15]:
tobin_2019['tobins_q'] = np.where(tobin_2019['total_asset'] > 0, ((tobin_2019['market_cap'] + tobin_2019['total_debt']) / tobin_2019['total_asset']), 0)
tobin_2019

,corp_code,corp_name,year,total_asset,asset_size,total_debt,debt_ratio,market_cap,tobins_q
0,365387,AJ네트웍스,2019,1803255225986.000000,28.220615,1455914070366.000000,0.807381,232706806150.000000,0.936429
1,125080,AK홀딩스,2019,4327964555085.000000,29.096118,2898025666994.000000,0.669605,458365610600.000000,0.775513
2,219097,BGF,2019,1657381460248.000000,28.136260,129126735585.000000,0.077910,536014029600.000000,0.401320
3,1263022,BGF리테일,2019,2172410899706.000000,28.406859,1549815137631.000000,0.713408,2929622067000.000000,2.061966
4,858364,BNK금융지주,2019,0.000000,0.000000,0.000000,0.000000,2496663984360.000000,0.000000
...,...,...,...,...,...,...,...,...,...
726,111421,휴니드테크놀러지스,2019,260965121649.000000,26.287653,145116828409.000000,0.556077,102623429050.000000,0.949323
727,362238,휴비스,2019,877170812678.000000,27.499968,481045043591.000000,0.548405,210105000000.000000,0.787931
728,156488,휴스틸,2019,579096387394.000000,27.084735,182538348103.000000,0.315212,76064246640.000000,0.446562
729,103176,흥국화재,2019,0.000000,0.000000,0.000000,0.000000,196903706925.000000,0.000000


In [16]:
def tobin_q(start_date = 2019, end_date = 2024):
    cadf_lst = []
    for year in range(2019, 2025):
        cadf_lst.append(pd.read_csv(f'data/Major_Shareholder/Major_Shareholder_correction/MSE_C_{year}.csv', index_col = 0))
    
    cadf_2019, cadf_2020, cadf_2021, cadf_2022, cadf_2023, cadf_2024 = cadf_lst
    
    rdf_lst = []
    for year in range(2019, 2025):
        rdf_lst.append(pd.read_csv(f'data/market_cap/market_cap_{year}.csv', encoding = 'utf-8', index_col = 0))
    
    rdf_2019, rdf_2020, rdf_2021, rdf_2022, rdf_2023, rdf_2024 = rdf_lst   
    
    tobin_lst = []
    for year, cadf, rdf in zip(range(start_date, end_date +1), cadf_lst, rdf_lst):
        tobin_df = cadf.merge(rdf, how = 'inner', on = ['corp_code', 'corp_name']).drop('stock_code', axis = 1)
        
        tobin_df['tobins_q'] = np.where(tobin_df['total_asset'] > 0, ((tobin_df['market_cap'] + tobin_df['total_debt']) / tobin_df['total_asset']), 0)
        
        tobin_lst.append(tobin_df)
        
    return tobin_lst

In [17]:
tobin_q_2019, tobin_q_2020, tobin_q_2021, tobin_q_2022, tobin_q_2023, tobin_q_2024 = tobin_q(start_date = 2019, end_date = 2024)

In [18]:
tobin_q_2019

,corp_code,corp_name,year,total_asset,asset_size,total_debt,debt_ratio,market_cap,tobins_q
0,365387,AJ네트웍스,2019,1803255225986.000000,28.220615,1455914070366.000000,0.807381,232706806150.000000,0.936429
1,125080,AK홀딩스,2019,4327964555085.000000,29.096118,2898025666994.000000,0.669605,458365610600.000000,0.775513
2,219097,BGF,2019,1657381460248.000000,28.136260,129126735585.000000,0.077910,536014029600.000000,0.401320
3,1263022,BGF리테일,2019,2172410899706.000000,28.406859,1549815137631.000000,0.713408,2929622067000.000000,2.061966
4,858364,BNK금융지주,2019,0.000000,0.000000,0.000000,0.000000,2496663984360.000000,0.000000
...,...,...,...,...,...,...,...,...,...
726,111421,휴니드테크놀러지스,2019,260965121649.000000,26.287653,145116828409.000000,0.556077,102623429050.000000,0.949323
727,362238,휴비스,2019,877170812678.000000,27.499968,481045043591.000000,0.548405,210105000000.000000,0.787931
728,156488,휴스틸,2019,579096387394.000000,27.084735,182538348103.000000,0.315212,76064246640.000000,0.446562
729,103176,흥국화재,2019,0.000000,0.000000,0.000000,0.000000,196903706925.000000,0.000000


In [19]:
tobin_q_lst = [tobin_q_2019, tobin_q_2020, tobin_q_2021, tobin_q_2022, tobin_q_2023, tobin_q_2024]

for year, df in zip(range(2019, 2025), tobin_q_lst):
    df.to_csv(f'data/market_cap/tobin_q/tobin_q_{year}.csv')

In [20]:
# df_foreign_2019 = stock.get_exhaustion_rates_of_foreign_investment("20191230")
# df_foreign_2019 = df_foreign_2019.reset_index()
# df_foreign_2019 = df_foreign_2019.drop(['상장주식수', '보유수량', '한도수량', '한도소진률'])